Alexander S. Lundervold, 19.04.22

# Introduction

When run, the transform component will grab the artifacts produced by our `ExampleGen` (our examples) and `SchemaGen` (the data schema). It will produce two artifacts: a computational graph (a TensorFlow graph) containing the preprocessing steps and the transformed examples stored as TFRecords (together with their statistics). 

This is what our pipeline will look like at the end of the notebook:

<img width=60% src="assets/pipeline_3.png">

The preprocessing steps we want our Transform component to perform will have to be specified in a preprocessing function `preprocessing_fn`. 

# Setup

Import basic libraries:

In [1]:
%matplotlib inline
import os
from pathlib import Path

Check whether we're running on Colab:

In [2]:
try:
    import colab
    colab=True
except:
    colab=False

Set up data directories:

In [3]:
if colab:
    from google.colab import drive
    drive.mount('./gdrive')
    DATA = Path('./gdrive/MyDrive/ColabData/petfinder-mini/csv')
else:
    NB_DIR = Path.cwd()
    DATA = NB_DIR/'..'/'data'/'petfinder-mini'/'csv'
    
SPLIT_DATA = DATA/'..'/'split_csv'

Install TFX and import components:

In [4]:
if colab:
    !pip install -U tfx

> If on Colab, restart the runtime after running the above cell

In [5]:
import tensorflow as tf
import tfx

In [10]:
from tfx.components import CsvExampleGen
from tfx.components import StatisticsGen
from tfx.components import SchemaGen
from tfx.components import ExampleValidator
from tfx.components import Transform

Set up the interactive context for running TFX components:

In [7]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

In [8]:
context = InteractiveContext()

# Recreate the previous pipeline

In [10]:
from tfx.components import CsvExampleGen
from tfx.components import StatisticsGen
from tfx.components import SchemaGen
from tfx.components import ExampleValidator
from tfx.components import Transform

In [11]:
# Generate examples
example_gen = CsvExampleGen(input_base=str(DATA)+'/')

# Generate statistics
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

# Automatic data schema (in a more realistic setting we would have 
# used a manually modified schema saved to disk)
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])

# Validate examples
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

### Execute the components

In [13]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [14]:
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [15]:
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [16]:
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [17]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Breed1',BYTES,required,,-
'Color1',STRING,required,,'Color1'
'Color2',STRING,required,,'Color2'
'Description',BYTES,required,,-
'FurLength',STRING,required,,'FurLength'
'Gender',STRING,required,,'Gender'
'Health',STRING,required,,'Health'
'MaturitySize',STRING,required,,'MaturitySize'
'Sterilized',STRING,required,,'Sterilized'


,Values
Domain,
'Color1',"'Black', 'Brown', 'Cream', 'Golden', 'Gray', 'White', 'Yellow'"
'Color2',"'Brown', 'Cream', 'Golden', 'Gray', 'No Color', 'White', 'Yellow'"
'FurLength',"'Long', 'Medium', 'Short'"
'Gender',"'Female', 'Male'"
'Health',"'Healthy', 'Minor Injury', 'Serious Injury'"
'MaturitySize',"'Large', 'Medium', 'Small'"
'Sterilized',"'No', 'Not Sure', 'Yes'"
'Type',"'Cat', 'Dog'"
'Vaccinated',"'No', 'Not Sure', 'Yes'"


In [18]:
context.show(example_validator.outputs['anomalies'])

Now we've recreated the pipeline from the previous notebook:

<img src='assets/pipeline_2.png'>

# The Transform component

The data preprocessing will be done using the Transform component (which is based on the standalone library [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started)).

In [19]:
from tfx.components import Transform

In [20]:
#?Transform

To perform our preprocessing, we need to take a closer look at the dataset we're using. We have to preprocess the numerical, categorical, ordinal, text and other features in data type-specific ways. This typically requires some manual work, as we have to make a bunch of decisions about how to represent each feature. (However, it's very possible to at least partially automate some of this. Some of you have for example seen the PyCaret library, and it's `setup` function that figures out workable preprocessing steps). 

> Add link. Add others?

In [21]:
import pandas as pd

In [22]:
df = pd.read_csv(DATA/'petfinder-mini.csv')

In [23]:
df.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He is energetic and playful. I rescued a couple of cats a few months ago but could not get them neutered in time as the clinic was fully scheduled. The result was this little kitty. I do not have enough space and funds to care for more cats in my household. Looking for responsible people to take over Nibble's care.,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartment. It was shaking so I had to bring it home to provide temporary care.,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,"Their pregnant mother was dumped by her irresponsible owner at the roadside near some shops in Subang Jaya. Gave birth to them at the roadside. They are all healthy and adorable puppies. Already dewormed, vaccinated and ready to go to a home. No tying or caging for long hours as guard dogs. However, it is acceptable to cage or tie for precautionary purposes. Interested to adopt pls call me.",7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience waiting for her good master, plz call or sms for more details if you really get interested, thanks!!",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,"This handsome yet cute boy is up for adoption. He is the most playful pal we've seen in our puppies. He loves to nibble on shoelaces , Chase you at such a young age. Imagine what a cute brat he will be when he grows. We are looking for a loving home for Hunter , one that will take care of him and give him the love that he needs. Please call urgently if you would like to adopt this cutie.",3,2


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11537 entries, 0 to 11536
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Type           11537 non-null  object
 1   Age            11537 non-null  int64 
 2   Breed1         11537 non-null  object
 3   Gender         11537 non-null  object
 4   Color1         11537 non-null  object
 5   Color2         11537 non-null  object
 6   MaturitySize   11537 non-null  object
 7   FurLength      11537 non-null  object
 8   Vaccinated     11537 non-null  object
 9   Sterilized     11537 non-null  object
 10  Health         11537 non-null  object
 11  Fee            11537 non-null  int64 
 12  Description    11528 non-null  object
 13  PhotoAmt       11537 non-null  int64 
 14  AdoptionSpeed  11537 non-null  int64 
dtypes: int64(4), object(11)
memory usage: 1.3+ MB


Here's some descriptions that we can gather from looking at the data and from reading the description on Kaggle [PetFinder.my Adoption Prediction](https://www.kaggle.com/c/petfinder-adoption-prediction/data) and in the TensorFlow catalog https://www.tensorflow.org/datasets/catalog/pet_finder. 

* `Type` - Type of animal (Cat, Dog)
* `Age` - Age of pet when listed, in months
* `Breed1` - Primary breed of pet (Refer to BreedLabels dictionary)
* `Gender` - Gender of pet (Male, Female)
* `Color1` - Color 1 of pet (Black, Brown, Cream, Gray, Golden, White, Yellow)
* `Color2` - Color 2 of pet ('White', 'Brown', 'No Color', 'Gray', 'Cream', 'Golden', 'Yellow')
* `MaturitySize` - Size at maturity (Small, Medium, Large)
* `FurLength` - Fur length (Short, Medium, Long)
* `Vaccinated` - Pet has been vaccinated (Yes, No, Not Sure)
* `Sterilized` - Pet has been spayed / neutered (Yes, No, Not Sure)
* `Health` - Health Condition (Healthy, Minor Injury, Serious Injury, Not Specified)
* `Fee` - Adoption fee (0 = Free)
* `Description` - Profile write-up for this pet. The primary language used is English, with some in Malay or Chinese.
* `PhotoAmt` - The total number of uploaded photos for each pet (numerical)
* `AdoptionSpeed` - Categorical speed of adoption. Lower is faster. This is the value to predict. 0 - Pet was adopted on the same day as it was listed; 1 - Pet was adopted between 1 and 7 days (1st week) after being listed; 2 - Pet was adopted between 8 and 30 days (1st month) after being listed; 3 - Pet was adopted between 31 and 90 days (2nd & 3rd month) after being listed; 4 - No adoption after 100 days of being listed.

| Column          | Pet description               | Feature type   | Data type |
| --------------- | ----------------------------- | -------------- | --------- |
| `Type`          | Type of animal (`Dog`, `Cat`) | Categorical    | String    |
| `Age`           | Age                           | Numerical      | Integer   |
| `Breed1`        | Primary breed                 | Categorical    | String    |
| `Color1`        | Color 1                       | Categorical    | String    |
| `Color2`        | Color 2                       | Categorical    | String    |
| `MaturitySize`  | Size at maturity              | Categorical    | String    |
| `FurLength`     | Fur length                    | Categorical    | String    |
| `Vaccinated`    | Pet has been vaccinated       | Categorical    | String    |
| `Sterilized`    | Pet has been sterilized       | Categorical    | String    |
| `Health`        | Health condition              | Categorical    | String    |
| `Fee`           | Adoption fee                  | Numerical      | Integer   |
| `Description`   | Profile write-up              | Text           | String    |
| `PhotoAmt`      | Total uploaded photos         | Numerical      | Integer   |
| `AdoptionSpeed` | Categorical speed of adoption | Classification | Integer   |

# Define a preprocessing function

NB: The transformation function specified in a separate module (.py file)

Ref Hapke & Nelson, Building Machine Learning Pipelines. [https://github.com/Building-ML-Pipelines/building-machine-learning-pipelines/blob/main/components/module.py](https://github.com/Building-ML-Pipelines/building-machine-learning-pipelines/blob/main/components/module.py)

In [25]:
pets_transform_file = 'pets_transforms.py'

In [2]:
#?%%writefile

In [27]:
df.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He is energetic and playful. I rescued a couple of cats a few months ago but could not get them neutered in time as the clinic was fully scheduled. The result was this little kitty. I do not have enough space and funds to care for more cats in my household. Looking for responsible people to take over Nibble's care.,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartment. It was shaking so I had to bring it home to provide temporary care.,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,"Their pregnant mother was dumped by her irresponsible owner at the roadside near some shops in Subang Jaya. Gave birth to them at the roadside. They are all healthy and adorable puppies. Already dewormed, vaccinated and ready to go to a home. No tying or caging for long hours as guard dogs. However, it is acceptable to cage or tie for precautionary purposes. Interested to adopt pls call me.",7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience waiting for her good master, plz call or sms for more details if you really get interested, thanks!!",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,"This handsome yet cute boy is up for adoption. He is the most playful pal we've seen in our puppies. He loves to nibble on shoelaces , Chase you at such a young age. Imagine what a cute brat he will be when he grows. We are looking for a loving home for Hunter , one that will take care of him and give him the love that he needs. Please call urgently if you would like to adopt this cutie.",3,2


In [31]:
%%writefile {pets_transform_file}

from typing import Union
import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = "AdoptionSpeed"

ONE_HOT_FEATURES = {
    'Type': 2,
    'Breed1': 166,
    'Gender': 2,
    'Color1': 7,
    'Color2': 7,
    'MaturitySize': 2,
    'FurLength': 3,
    'Vaccinated': 3,
    'Sterilized': 3,
    'Health': 3
    
}

NUMERICAL_FEATURES = [
    'Age',
    'Fee',
    'PhotoAmt' 
]

TEXT_FEATURES = {'Description': None}


def _transformed_name(key:str) -> str:
    return key + "_xf"



def _convert_num_to_one_hot(label_tensor: tf.Tensor, num_labels: int = 2) -> tf.Tensor:
    """
    Convert a label (0 or 1) into a one-hot vector
    Args:
        int: label_tensor (0 or 1)
    Returns
        label tensor
    """
    one_hot_tensor = tf.one_hot(label_tensor, num_labels)
    return tf.reshape(one_hot_tensor, [-1, num_labels])

def _fill_in_missing(x: Union[tf.Tensor, tf.SparseTensor]) -> tf.Tensor:
    """Replace missing values in a SparseTensor.
    Fills in missing values of `x` with '' or 0, and converts to a
    dense tensor.
    Args:
      x: A `SparseTensor` of rank 2.  Its dense shape should have
        size at most 1 in the second dimension.
    Returns:
      A rank 1 tensor where missing values of `x` have been filled in.
    """
    if isinstance(x, tf.sparse.SparseTensor):
        default_value = "" if x.dtype == tf.string else 0
        x = tf.sparse.to_dense(
            tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
            default_value,
        )
    return tf.squeeze(x, axis=1)


def preprocessing_fn(inputs: tf.Tensor) -> tf.Tensor:
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    
    outputs = {}

    for key in ONE_HOT_FEATURES:
        dim = ONE_HOT_FEATURES[key]
        int_value = tft.compute_and_apply_vocabulary(
            _fill_in_missing(inputs[key]), top_k=dim+1
        )
        outputs[_transformed_name(key)] = _convert_num_to_one_hot(
            int_value, num_labels=dim+1
        )

    for key in NUMERICAL_FEATURES:
        # Scale these features to the z-score.
        outputs[_transformed_name(key)] = tft.scale_to_z_score(inputs[key])
            
    for key in TEXT_FEATURES.keys():
        outputs[_transformed_name(key)] = _fill_in_missing(inputs[key])

        
    outputs[_transformed_name(LABEL_KEY)] = _fill_in_missing(inputs[LABEL_KEY])

    return outputs

Overwriting pets_transforms.py


In [32]:
import os

In [33]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(pets_transform_file))

In [34]:
context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-04-19T09_17_41.582665-a5nbogne/Transform/transform_graph/5/.temp_path/tftransform_tmp/e6a8705bfc74425481a81e2a29e125b7/assets


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-04-19T09_17_41.582665-a5nbogne/Transform/transform_graph/5/.temp_path/tftransform_tmp/e6a8705bfc74425481a81e2a29e125b7/assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

### Inspect some transformed examples

In [83]:
train_uri = transform.outputs['transformed_examples'].get()[0].uri + '/Split-train'
train_uri

'/tmp/tfx-interactive-2022-04-15T15_14_53.842185-ckevaf8u/Transform/transformed_examples/13/Split-train'

In [84]:
os.listdir(train_uri)

['transformed_examples-00000-of-00001.gz']

In [86]:
transformed_dataset = tf.data.TFRecordDataset(train_uri+'/transformed_examples-00000-of-00001.gz', 
                                              compression_type="GZIP")

In [87]:
transformed_dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

Here is the first record records:

In [92]:
for tfrecord in transformed_dataset.take(1):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    print(example)
    print("#"*40)
    print("#"*40)
    print()

features {
  feature {
    key: "AdoptionSpeed_xf"
    value {
      int64_list {
        value: 2
      }
    }
  }
  feature {
    key: "Age_xf"
    value {
      float_list {
        value: -0.4461670219898224
      }
    }
  }
  feature {
    key: "Breed1_xf"
    value {
      float_list {
        value: 0.0
        value: 0.0
        value: 0.0
        value: 1.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
  

### The transform graph artifact

In [72]:
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

In [73]:
transform_graph_uri

'/tmp/tfx-interactive-2022-04-15T15_14_53.842185-ckevaf8u/Transform/transform_graph/13'

In [75]:
os.listdir(transform_graph_uri)

['transform_fn', 'transformed_metadata', 'metadata']

In [77]:
os.listdir(transform_graph_uri + '/metadata')

['schema.pbtxt']

In [78]:
os.listdir(transform_graph_uri + '/transformed_metadata')

['schema.pbtxt', 'asset_map']

In [76]:
os.listdir(transform_graph_uri + '/transform_fn')

['saved_model.pb', 'variables', 'assets']

# What have we done so far?

Here's our current pipeline:

<img width=60% src="assets/pipeline_3.png">

# What's next?

We now have a pipeline that ingests data, computes statistics, generates a data schema, applies the schema to validate examples, and preprocesses the examples. Next, we'll look at how to do the actual **training of machine learning models**.

The training will be done by a TFX **Trainer** component, which consumes the transformed examples from the `Transform` component, and the data schema. 

We'll end up with the following pipeline:

<img width=100% src="assets/pipeline_4.png">